# Lab02: Frequent itemset mining

- Student ID: 20120113
- Student name: Lê Nguyên Khang

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [144]:
from collections import defaultdict

### Read data
First we have to read data from database

In [145]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    
    return data, s

### Tree Projection

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**

In [146]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    ret = set(a + b)
    ret = list(sorted(ret))
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below?

        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()

    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}

        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}

            search_space[item]['itemset'] = [item]
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.

            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.

            search_space[item]['support'] = support

            tree[item] = {}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.

            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''

        return tree, search_space

    def computeItemsetSupport(self, itemset):

        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        pat = frozenset(itemset)
        support = 0 
        for d in self.data:
            ds = frozenset(self.data[d])
            if pat.issubset(ds):
                support += 1
        return support

    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)


    def prune(self, k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO
        for itemset in search_space.keys():
            if search_space[itemset]['pruned']: continue
            if search_space[itemset]['support'] < self.minSup:
                search_space[itemset]['pruned'] = True
            else:
                self.L[k].append(search_space[itemset]['itemset'])

    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue

            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                # TODO:
                # First create newset using join set
                newset = tuple(joinset(search_space[a]['itemset'], search_space[b]['itemset']))

                # Second add newset to search_space
                sub_search_space[newset] = {}
                sub_search_space[newset]['itemset'] = newset
                sub_search_space[newset]['support'] = self.computeItemsetSupport(newset)
                sub_search_space[newset]['pruned'] = False
                sub_tree[newset] = {}
                
            #  Generate sub_search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)

    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L


Ok, let's test on a typical dataset `chess`.

In [147]:
data, s= readData('chess.txt')
    

In [148]:
a=TP(data=data,s=s, minSup=3000)
print(a.miningResults())

{1: [[48], [56], [66], [34], [62], [7], [36], [60], [40], [29], [52], [58]], 2: [(48, 52), (48, 58), (29, 56), (52, 56), (56, 58), (60, 66), (29, 66), (52, 66), (58, 66), (34, 40), (29, 34), (34, 52), (34, 58), (60, 62), (40, 62), (29, 62), (52, 62), (58, 62), (7, 60), (7, 40), (7, 29), (7, 52), (7, 58), (36, 60), (36, 40), (29, 36), (36, 52), (36, 58), (40, 60), (29, 60), (52, 60), (58, 60), (29, 40), (40, 52), (40, 58), (29, 52), (29, 58), (52, 58)], 3: [(48, 52, 58), (29, 52, 56), (29, 56, 58), (52, 56, 58), (29, 60, 66), (52, 60, 66), (58, 60, 66), (29, 52, 66), (29, 58, 66), (52, 58, 66), (29, 34, 40), (34, 40, 52), (34, 40, 58), (29, 34, 52), (29, 34, 58), (34, 52, 58), (29, 60, 62), (52, 60, 62), (58, 60, 62), (29, 40, 62), (40, 52, 62), (40, 58, 62), (29, 52, 62), (29, 58, 62), (52, 58, 62), (7, 40, 60), (7, 29, 60), (7, 52, 60), (7, 58, 60), (7, 29, 40), (7, 40, 52), (7, 40, 58), (7, 29, 52), (7, 29, 58), (7, 52, 58), (36, 40, 60), (29, 36, 60), (36, 52, 60), (36, 58, 60), (29

### Answer questions here:
**Why don't we compute support of items while reading data?**
- Đầu tiên chúng ta tính toán độ hổ trợ của các items trong khi đọc dữ liệu để từ đó biết được items nào có khả năng tạo tập phổ biết lớn hơn. Sau đó khi mỗi itemset được tạo thành ta mới tính support của nó vì các itemset mới được tạo dựa vào tập ứng viên vượt qua ngưỡng support trước đó, việc này làm giảm các itemset được tạo thành để tính support và làm cho thuật toán hiệu quả hơn. Bằng cách này giúp thuật toán thực hiện được theo cách cân bằng giữa độ hiệu quả và độ chính xác.

**why should we do sort**
- Việc sắp xếp các mục dựa trên tần suất xuất hiện của chúng trong các giao dịch của tập dữ liệu là một bước quan trọng để cải thiện hiệu quả của thuật toán. Khi sắp xếp cho phép thuật toán tập trung đầu tiên vào các mục thường xuyên nhất, có khả năng xảy ra cùng nhau trong nhiều giao dịch, trước khi xem xét các mục ít thường xuyên hơn. Ngoài ra, khi sắp xếp thuật toán có thể thực hiện cắt tỉa, đây là quá trình loại bỏ các tập mục ứng viên không thể phổ biến dựa trên các giá trị hỗ trợ của các tập con của chúng làm giảm không gian tìm kiếm và tăng tốc thuật toán.

**study about python set and its advantages ?**
- Loại bỏ các mục trùng lặp: Khi có các itemset trùng lặp, tập set sẽ cho ta biết key của itemset ấy đã tồn tại hay chưa.
- Truy xuất nhanh: Khi cần gọi kiểm tra itemset có support bao nhiêu? đã được prune hay chưa? được lưu trước đó thì dùng tập set làm việc này rất nhanh.
- Hiệu quả về bộ nhớ: Vì tập set có thể tiết kiệm bộ nhớ hơn so với danh sách khi xử lý các tập dữ liệu lớn.
- Tổ chức lưu trữ itemset theo tree, search_space hiệu quả linh hoạt có thể mở rộng tùy ý dễ dàng đồng thời có thể kết hợp với kiểu list trong quá trình lưu trữ. 
- Các phép toán giao và hợp: Các tập set trong Python cũng có các phép toán giao và hợp hiệu quả, được sử dụng trong thuật toán để tạo ra các tập ứng viên có kích thước tăng dần.

-> Thuật toán có sự cải thiện đáng kể về hiệu quả và hiệu xuất khi sử dụng tập set.

**After finish implementing the algorithm tell me why should you use this? Instead of delete item directly from search_space and tree.**
- Khả năng mất thông tin: Khi bạn xóa một mục khỏi không gian tìm kiếm và cây có thể sẽ mất thông tin về hỗ trợ của mục đó và mối quan hệ của mục đó với các mục khác trong tập dữ liệu. Điều này có thể khiến việc phân tích kết quả của thuật toán và xác định các mẫu thú vị trở nên khó khăn và có thể có sai sót bên trong.
- Khả năng xảy ra lỗi: Việc xóa các mục trực tiếp khỏi không gian tìm kiếm và cây có thể dễ bị lỗi, đặc biệt với dữ liệu có kích thước lớn. Điều này có thể dẫn đến lỗi và kết quả không chính xác cho thuật toán.
- Sau khi một mục bị xóa khỏi không gian và cây tìm kiếm, mục ấy có thể khó đảo ngược thay đổi và khôi phục trạng thái ban đầu. Đây có thể là vấn đề khi ta cần sửa đổi thuật toán hoặc phân tích kết quả theo những cách khác nhau.
- Giảm tính linh hoạt: Bằng cách xóa trực tiếp các mục, có thể sẽ bị hạn chế tính linh hoạt của thuật toán để thích ứng với các bộ dữ liệu và các yêu cầu phân tích khác nhau. Đây có thể là vấn đề nếu khi ta tái sử dụng thuật toán cho các dự án hoặc bộ dữ liệu khác nhau.
- Ngược lại, bằng cách triển khai thuật toán mà không xóa trực tiếp các mục khỏi không gian và cây tìm kiếm, ta có thể bảo toàn thông tin đầy đủ về tập dữ liệu và mối quan hệ của nó giữa các mục. Điều này có thể giúp phân tích kết quả và xác định các tập phổ biến trở nên dễ dàng hơn.

**Apriori algorithm and Tree Projection, tell me the differences of two algorithms.**
- Thuật toán Apriori sử dụng cách tiếp cận "từ dưới lên", trong đó thuật toán bắt đầu bằng cách xác định tất cả các mục riêng lẻ phổ biến (tập mục có kích thước 1) trong tập dữ liệu, sau đó lặp lại tạo các tập mục lớn hơn bằng cách nối các tập mục phổ biến từ lần lặp trước. Thuật toán cắt bớt các tập phổ biến ở mỗi lần lặp để giảm không gian tìm kiếm và cải thiện hiệu suất. Thuật toán phải quét toàn bộ tập dữ liệu nhiều lần cho mỗi lần lặp lại, điều này có thể tốn kém về mặt tính toán đối với các tập dữ liệu lớn.

- Mặt khác, thuật toán Tree Projection là một cách tiếp cận "từ trên xuống" hoạt động bằng cách xây dựng một cây tiền tố từ tập dữ liệu. Cây tiền tố sau đó được chiếu để xác định các tập phổ biến bằng cách khám phá đệ quy các nút của cây. Trong mỗi lời gọi đệ quy, thuật toán sẽ kiểm tra xem nút hiện tại có thể được mở rộng thành tập phổ biến hay không và cắt bớt cây con nếu không. Thuật toán chỉ quét tập dữ liệu một lần để xây dựng cây tiền tố, đây có thể là cách tiếp cận hiệu quả hơn cho các tập dữ liệu lớn.



# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). 

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  

In [149]:

def readData_churn(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data = {}
    s = defaultdict(lambda: 0)
    with open(path,'rt') as f:
        tid = 0
        for line in f:
            if tid == 0:
                id = line.strip('\n?').split(',')
            else:
                itemset = line.strip(".\n").split(',')
                row = []
                i = 0
                for item in itemset:  
                    value = id[i] + ':' + item
                    s[value]+=1
                    row.append(value)
                    i += 1
                data[tid]= row

            tid+=1
    return data, s

In [150]:
data2, s2 = readData_churn('churn.txt')
a2 = TP(data=data2, s=s2, minSup=2000)
print(a2.miningResults())

{1: [['VMail Plan:no'], ['VMail Message:0'], ['Churn:False'], ["Int'l Plan:no"]], 2: [('VMail Message:0', 'VMail Plan:no'), ('Churn:False', 'VMail Plan:no'), ("Int'l Plan:no", 'VMail Plan:no'), ('Churn:False', 'VMail Message:0'), ("Int'l Plan:no", 'VMail Message:0'), ('Churn:False', "Int'l Plan:no")], 3: [('Churn:False', 'VMail Message:0', 'VMail Plan:no'), ("Int'l Plan:no", 'VMail Message:0', 'VMail Plan:no')], 4: []}


# 4 References

Feel free to send questions to my email address: nnduc@fit.hcmus.edu.vn
